#2개의 층
케라스 API를 사용하여 패션 MNIST 데이터셋을 불러온다.


In [2]:
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


다음으로 이미지의 픽셀값을 0에서 255범위에서 0~1 사이로 변환하고, 28*28 크기의 2차원 배열을 784 크기의 1차원 배열로 펼친다. 그 후 훈련 세트와 검증 세트로 나눈다.

In [3]:
from sklearn.model_selection import train_test_split
train_scaled = train_input/255.0
train_scaled = train_scaled.reshape(-1,28*28)
train_scaled, val_scaled, train_target, cal_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state = 42)



*   은닉층 : 입력층과 출력층 사이에 있는 모든 층

은닉층에는활성화 함수라는 것이 있다.



*   활성화 함수 : 신경망 층의 선형 방정식의 계산 갑셍 적용하는 함수







다음으로 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만든다.

In [6]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))
dense2 = keras.layers.Dense(10, activation='softmax')

dense1은 은닉층이고 100개의 뉴런을 가진 밀집층이다. 활성화 함수는 sigmoid이고 input_shape 매개변수에서 입력의 크기를 (784,)로 지정했다.      
이때 뉴런의 개수를 정하는 데에는 특별한 기준이 없다. 많은 경험을 통해 알아가야 한다.

한가지 제약사항이 있다. 은닉층의 뉴런은 적어도 출력층의 뉴런보다 많게 만들어야 한다.

dense2는 출력층이다. 10개의 클래스를 분류하므로 10개의 뉴런을 두고, 활성화 함수는 소프트맥스 함수로 지정했다.

#인공 신경망 만들기
이제 dense1, dense2 객체를 Sequential 클래스에 추가하여 심층 신경망을 만들어본다.

In [ ]:
model = keras.Sequential([dense1, dense2])

여러개의 층을 추가하기 위해 dense1, dense2를 리스트로 만들어 전달한다. 이때 출력층을 가장 마지막에 두어야 한다는 것이다.

케라스 모델의 summary 메서드를 호출해 층에 대한 정보를 확인할 수 있다.

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


처음 모델의 이름이 나온다. 그 다음 모델에 들어있는 층이 순서대로 나열된다. 이 순서는 맨 처음 추가한 은닉층에서 출력층의 순서대로 출력된다.

맨 처음의 출력 크기는 (None, 100)이다. 첫번재 자원은 샘플의 개수를 뜻하고 두번째는 출력 개수이다.

 미니 배치 경사 하강법을 사용하여 데이터를 모두 한번에 사용하지 않고 잘게 나누어 여러 번에 걸쳐 경사 하강법 단계를 수행하기 때문이다.

따라서 샘플마다 784개의 픽셀 값이 은닉층을 통과하면서 100개의 특성으로 압축된 것이다.

##층을 추가하는 다른 방법
층을 추가하는 다은 방법으로는 Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 것이다.

In [10]:
model = keras.Sequential([
    keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
    keras.layers.Dense(10, activation='softmax', name='output')],
                         name='패션 MNIST 모델')

이렇게 작업하면 한 번에 층을 알아볼 수 있다는 장점이 있다. 이번에는 name 매개변수로 모델의 이름을 지정햇다. 또한 은닉층과 출력층 또한 이름을 붙여줬다.

In [11]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


동일한 Dense 층과 파라미터 개수를 확인할 수 있다. 또한 name 매개변수로 이름을 지정해 층을 구분하기 쉬워졌다.

또 다른 방법으로 add() 메서드를 사용하는 방법이 있다.  

In [14]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10,activation='softmax'))

위와 같이 add메서드를 사용하면 한번에 추가되는 층을 볼수 있고, 동적으로 층을 선택하여 추가할 수도 있다.

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


처음과 같은 결과를 얻을 수 있다.

이제 모델을 훈련해본다 compile() 메서드를 사용하고 fit()의 매개변수 epochs로 5번 훈련한다.

In [17]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5678 - accuracy: 0.8075
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4102 - accuracy: 0.8542
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3765 - accuracy: 0.8645
Epoch 4/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3537 - accuracy: 0.8720
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3359 - accuracy: 0.8781


#렐루 함수
초기 인공 신경망의 은닉층에 많이 사용된 활성화 함수는 시그모이드 함수였다. 하지만 이 함수에는 단점이 있다. 함수가 왼쪽 끝과 오른쪽 끝으로 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다는 것이다.

이를 개선하기 위해 렐루 함수가 제안되었다.


*   렐루 함수 : 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고 음수일 경우에는 0으로 만드는 함수이다.

렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다고 알려져 있다.



##Flatten 클래스
Flatten 클래스느느 배치 차원을 제외하고 나머지 읿력 차원을 모두 일렬로 펼치는 역할을 한다. 인공 신경망의 성능을 위해 기여하는 바는 없지만 입력층과 은닉층 사이에 추가하기 때문에 층이라 부른다.

In [19]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

첫 번째 Dense 층에 있던 input_shape 매개변수를 Flatten 층으로 옮겼다. 또한 첫 번재 Dense 층의 활성화 함수를 relu로 바꿨다.

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_8 (Dense)             (None, 100)               78500     
                                                                 
 dense_9 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


flatten 층의 파라미터는 0개이다. 케라스의 Flatten 층을 신경망 모델에 추가하면 입력 값의 차원을 짐작할 수 있는 것이 장점이다. 위의 출력을 보면 (NOne, 784)를 통해 784개의 입력을 알 수 있다.

다시 훈련 데이터를 준비해서 모델을 훈련해본다. 이 때 앞선 전처리와 다르게 reshape 메서드를 이용해 배열의 크기를 바꾸지 않는다.

In [24]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input/255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state = 42)

In [22]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5324 - accuracy: 0.8122
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3937 - accuracy: 0.8587
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3571 - accuracy: 0.8720
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3342 - accuracy: 0.8791
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3194 - accuracy: 0.8837


시그모이드 함수보다 성능이 조금 향상됨을 볼 수 있다. 검증 세트에서의 성능 또한 확인해본다.

In [25]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3439 - accuracy: 0.8821


[0.34393155574798584, 0.8820833563804626]

#옵티마이저

3장에서 하이퍼 파라미터는 모델이 학습하지 않아 사람이 지정해주어야 하는 파라미터라고 했다. 신경망에는 하이퍼 파라미터가 많다.



추가할 은닉층의 개수 또한 우리가 지정해야 하는 하이퍼파아미터이다. 은닉층의 뉴런 개수 또한 하이퍼 파라미터이다.

케라스는 기본적으로 미니배치 경사 하강법을 사용하고, 미니배치의 개수는 기본적으로 32개지만 이 수치 또한 fit() 메서드의 fatch_size를 이용해 변경할 수 있고, 이 값 또한 하이퍼파라미터이다. epochs 또한 그렇다.

compile() 메서드에서는 케라스의 기본 경사 하강법 알고리즘인 RMSprop를 사용했다. 케라스는 다양한 종류의 경사 하강법 알고리즘을 제공하는데 이것들을 ***옵티마이저***라고 한다.

가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD이다. 이름은 SGD이지만 기본적으로 미니 배치를 사용한다.

SGD 옵티마이저를 사용하려면 compile 메서드의 optimizer 매개변수 를 'sgd'라고 지정한다.

In [28]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

이 옵티마이저는 tensorflow.keras.optimizers 패키지 아래의 SGD 클래스로 구현되어 있다.      
다음의 코드는 위의 코드와 일치하는 코드이다.

In [30]:
sgd = keras.optimizers.SGD()
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

SGD 클래스의 학습률 기본값은 0.01 이다. 이 때 이름을 바꾸고 싶다면 learning_rate 매개변수를 지정하여 사용한다.

In [32]:
sgd = keras.optimizers.SGD(learning_rate=0.1)

이 외에도 많은 옵티마이저들이 있다.

기본 경사 하강법 옵티마이저는 모두 SGD클래스에서 제공한다. SGD 클래스의 momentum 매개변수의 기본값은 0이다. 이를 0보다 큰 값으로 지정하면 ***모멘텀 최적화***를 사용한다.


*   모멘텀 가속화 : 이전의 그레이디언트를 가속도처럼 사용하는 방식

보통 momentum 매개변수는 0.9 이상을 지정한다.




SGD 클래스의 nesterov 매개변수를 기본값 False 에서 True로 바꾸면 ***네스테로프 모멘텀 최적화***를 사용한다.

In [35]:
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현한다. 대부분은 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법보다 더 나은 성능을 제공한다.



*   ***적응적 학습률*** : 모델이 최적점에 가까이 갈수록 학습률을 낮추는 것

이러한 방식들을 통해 학습률 매개변수를 튜닝하는 수고를 덜 수 있다.



적응적 학습률을 사용하는 대표적인 옵티마이저는 RMSprop이다.    
complie() 메서드의 optimizer 매개변수에 'adagrad'와 'rmsprop'로 지정할 수 있다.     
optimizer 매개변수의 기본값은 rmsprop이다.

In [37]:
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=adagrad, loss='sparse_categorival_crossentropy', metrics='accuracy')

RMSprop의 방식은 다음과 같다.

In [40]:
rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=rmsprop, loss='sparse_categorival_crossentropy', metrics='accuracy')

모멘텀 최적화와 RMSprop의 장점을 접목한 것이 Adam이다. 이 3개의 클래스는 learning_rate 매개변수이 기본값으로 모두 0.001을 사용한다.

아래에서는 Adam 클래스의 매개변수 기본값을 사용해여 패선 MNIST를 훈현한다.     
모델 생성부터 다시 한다.

In [44]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dense(10,activation='softmax'))

In [46]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5188 - accuracy: 0.8177
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3924 - accuracy: 0.8590
Epoch 3/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3513 - accuracy: 0.8738
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3252 - accuracy: 0.8827
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3052 - accuracy: 0.8888


RMSprop를 사용했을 때와 거의 같은 성능을 보여둔다. 아래는 검증 세트에서의 성능이다.

In [47]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3741 - accuracy: 0.8672


[0.3741055428981781, 0.8671666383743286]